# Use GloVe embeddings of word meanings

In [1]:
import pandas as pd
from toxic_funcs import Cleaner
import numpy as np

In [2]:
#reading only first n for testing purposes
n = 10000
df = pd.read_csv("data/train.csv").set_index('id')
df = df.head(n)

In [3]:
#running through the custom cleaner
from re import subn

clean_comments = []
for comment in df.comment_text.values:
    cleaned, _ = subn('[(){[]}]:;', '', Cleaner(comment))
    clean_comments.append(cleaned)


In [4]:
#slapping it onto df

df['cleaned'] = clean_comments

In [5]:
import os
import sys

TEXT_DATA_DIR = "../TEXT_DATA_DIR"

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids

texts = df.cleaned.values
labels_index = {0: 0, 1:1}
labels = df.toxic.values


In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT =.3

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]


Using TensorFlow backend.
/Users/nicksteinhauser/anaconda/envs/py36/lib/python3.6/site-packages/keras/preprocessing/text.py:157: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 36625 unique tokens.
Shape of data tensor: (10000, 1000)
Shape of label tensor: (10000, 2)


In [7]:
embeddings_index = {}
GLOVE_DIR = os.path.join('..','TEXT_DATA_DIR', 'glove.6B')
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Found 400000 word vectors.


In [8]:
EMBEDDING_DIM  = 100

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [9]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)


In [ ]:
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense
from keras.models import Model

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(256, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(12, activation='relu')(x)



preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

# happy learning!
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=2, batch_size=128)


Train on 7000 samples, validate on 3000 samples
Epoch 1/2
7000/7000 [==============================] - 608s 87ms/step - loss: 0.3132 - acc: 0.8899 - val_loss: 0.2567 - val_acc: 0.9003
Epoch 2/2
1536/7000 [=====>........................] - ETA: 6:37 - loss: 0.2848 - acc: 0.8991

In [12]:
model_loss, model_accuracy = model.evaluate(x_val, y_val, verbose=2)
print(f"Normal Neural Network Test - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network Test - Loss: 0.33688091425100963, Accuracy: 0.9099999998410543
